In [1]:
import pandas as pd
import numpy as np

raw_df = pd.read_csv('/Users/cynding/Desktop/Brandeis 2022 Spring/HS 256F - Healthcare Data Analytics/HW3/MCBSpuf2016/puf2016.csv')

/Users/cynding/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (34,48,53,66,85,86,87,89,103,104,112,113,114,130,132,134,136,137,138,139,140,141,142,143,144,145,146,147,160,165,166,167,170,171,183,186,191,192,193,194,196,197,201,209,211,216,218,220,221,222,223,226,232,248,249,254,256,257,263,264,265,266,267,269,270,271,281,282,283,284,289,290,291,292,293,294,295,296,298,300,302,309,310) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
raw_df.head()

,PUF_ID,SVYYR,ADM_H_ENT,ADM_H_MEDSTA,ADM_H_GHPSW,ADM_H_PDRS,ADM_OP_MDCD,ADM_PARTD,ADM_ACOFLAG,ADM_FFS_FLAG_YR,...,EEYRS091,EEYRS092,EEYRS093,EEYRS094,EEYRS095,EEYRS096,EEYRS097,EEYRS098,EEYRS099,EEYRS100
0,16000001,2016,2,3,2,1.0,2,1,1,3,...,1730.863696,315.422808,1772.439746,2014.649096,325.924565,1515.141757,313.119607,2046.638105,393.058043,380.735043
1,16000002,2016,1,1,2,1.0,1,2,2,3,...,6448.706281,24524.935702,4837.843981,7129.014867,29151.731944,29396.561818,5072.077989,27137.197856,5097.901657,4878.055919
2,16000006,2016,2,1,1,1.0,1,1,2,1,...,3589.669228,499.620974,517.477715,2904.884066,2866.824292,580.906592,535.377989,3361.963502,3010.362589,497.151719
3,16000008,2016,2,1,1,1.0,1,1,2,1,...,3740.417968,783.611754,4256.555251,824.458231,4553.515534,665.623951,767.555640,680.624207,3780.704442,671.883483
4,16000012,2016,2,1,2,1.0,1,1,2,3,...,4104.799801,749.123388,3521.030891,713.109543,3600.575591,705.059726,3831.571491,650.836074,4093.686322,768.398950


In [3]:
raw_df.columns

Index(['PUF_ID', 'SVYYR', 'ADM_H_ENT', 'ADM_H_MEDSTA', 'ADM_H_GHPSW',
       'ADM_H_PDRS', 'ADM_OP_MDCD', 'ADM_PARTD', 'ADM_ACOFLAG',
       'ADM_FFS_FLAG_YR',
       ...
       'EEYRS091', 'EEYRS092', 'EEYRS093', 'EEYRS094', 'EEYRS095', 'EEYRS096',
       'EEYRS097', 'EEYRS098', 'EEYRS099', 'EEYRS100'],
      dtype='object', length=412)

### Q6 Loneliness & Health

In [4]:
# Filter 65+ agegroup
survey_df = raw_df[raw_df['ADM_H_MEDSTA']==1].reset_index()
survey_df.shape

(10629, 413)

In [5]:
print(survey_df['HLT_GENHELTH'].unique())
print(survey_df['DEM_MARSTA'].unique())

['3' '1' '2' '4' '5' 'D' nan 'R']
['1' '2' '4' '3' 'D' 'R' 4 2 1 3]


In [6]:
survey_df.dropna(subset = ['HLT_GENHELTH','DEM_MARSTA'], inplace=True)
survey_df.shape

(10623, 413)

In [7]:
survey_df = survey_df[(survey_df['HLT_GENHELTH'] != 'D') & 
                     (survey_df['HLT_GENHELTH'] != 'R') &
                     (survey_df['DEM_MARSTA'] != 'D') &
                     (survey_df['DEM_MARSTA'] != 'R')]

Recode HLT_GENHELTH to HealthCondition

In [8]:
survey_df['HLT_GENHELTH'] = survey_df['HLT_GENHELTH'].astype(int)
survey_df['HealthCondition'] = np.where(survey_df['HLT_GENHELTH'] >= 4, 'PoorFairHealth', 'GoodHealth')

Recode DEM_MARSTA to Loneliness

In [9]:
survey_df['DEM_MARSTA'] = survey_df['DEM_MARSTA'].astype(int)
survey_df['Loneliness'] = np.where(survey_df['DEM_MARSTA'] == 1, 'WithFamily', 'LivingAlone')

In [25]:
crosstab1 = pd.crosstab(survey_df['Loneliness'],survey_df['HealthCondition'])
print(crosstab1)
print(crosstab1.iloc[0,0],crosstab1.iloc[0,1],crosstab1.iloc[1,0],crosstab1.iloc[1,1])

HealthCondition  GoodHealth  PoorFairHealth
Loneliness                                 
LivingAlone            4033            1049
WithFamily             4590             895
4033 1049 4590 895


In [27]:
from scipy.stats import fisher_exact
oddsratio,pvalue=fisher_exact([[crosstab1.iloc[0,0],crosstab1.iloc[0,1]], [crosstab1.iloc[1,0],crosstab1.iloc[1,1]]])
pvalue

1.138422792013177e-08

### Q7 Loneliness & Risk of depression

In [11]:
survey_df['Depression'] = np.where(survey_df['HLT_OCDEPRSS'] == '1', 'Yes', 'No')

In [30]:
crosstab2 = pd.crosstab(survey_df['Loneliness'],survey_df['Depression'])

In [31]:
oddsratio,pvalue=fisher_exact([[crosstab2.iloc[0,0],crosstab2.iloc[0,1]], [crosstab2.iloc[1,0],crosstab2.iloc[1,1]]])
pvalue

1.4913376578109423e-16

Fisher exact test
https://www.youtube.com/watch?v=gt9_x_rhLR8

In [34]:
oddsratio,pvalue=fisher_exact([[24,7], [19,16]],alternative='greater')
pvalue

0.04284138812876822